In [266]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary


import time


In [267]:
chrome_path = "./chromedriver"
#options = webdriver.ChromeOptions()
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
#options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
#options.add_argument("--user-agent=" + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')
driver = webdriver.Chrome(chrome_path, options=options)

In [268]:
def filterWords(words):
    newWords = []
    for word in words:
        word = word.lower()
        word = word.replace("!", "")
        word = word.replace("?", "")
        word = word.replace(":", "")
        word = word.replace(";", "")
        word = word.replace(".", "")
        word = word.replace(",", "")
        word = word.replace('\'', "")
        word = word.replace('\"', "")
        #words below are ones where we want to connect together to ensure these words are joined
        word = word.replace("trumps", "trump")
        word = word.replace("russian", "russia")
        word = word.replace("russians", "russia")
        word = word.replace("dems", "democrat")
        word = word.replace("democrats", "democrat")
        word = word.replace("republicans", "republican")
        word = word.replace("libs", "liberal")
        word = word.replace("liberals", "liberal")
        word = word.replace("conservatives", "conservative")
        word = word.replace("taxes", "tax")
        word = word.replace("socialists", "socialist")
        word = word.replace("nationalists", "nationalist")
        word = word.replace("globalists", "globalist")
        word = word.replace("corrupted", "corrupt")
        newWords.append(word)
    return newWords

In [269]:
# Scrape Huffington (limited history = 15 pages --> 260 articles)
# Set up arrays 
article_title = []
article_URL = []

def scrapeTest(url):
    articlesDF = []
    wordsDF = []
    html_doc = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    articleDataFrame = []
    wordDataFrame = [] #this is going to be the unorganized word DF

    articleList = soup.find('div', {'class': 'a-page__column--center'})
    articleList = articleList.find("div", {"class": "zone__content"})
    articles = articleList.findAll("div", {"class": "card"})

    for article in articles:
            linkHTML = article.find("a", {"class": "card__link yr-card-headline"})
            link = linkHTML['href']
            #commentLink = link[link.find('us_'):]
            print(link)
            print(commentLink)

            linkURL = "https://www.huffingtonpost.com" + link
            #commentURL = "https://m.huffpost.com/us/entry/" + commentLink + "/comments"
            print(linkURL)
            print(commentURL)
          
            articleTitle = article.find('div', {'class': 'card__headline__text'}).text
            print(articleTitle)
            wordsInTitle = articleTitle.split()
            wordsInTitle = filterWords(wordsInTitle)
            
            begin_url = 'https://www.huffingtonpost.com/entry/'
            cleaned_title = linkURL[linkURL.index(begin_url)+len(begin_url):linkURL.index("_")]
            cleaned_title = re.sub("-", " ", cleaned_title)
            
            #html_docComments = requests.get(commentURL,  headers={'User-Agent': 'Mozilla/5.0'}, allow_redirects=False)
            #commentSoup = BeautifulSoup(html_docComments.text, 'html.parser')
            #print(commentSoup)
            
            driver.get(linkURL)
            originalSoup = BeautifulSoup(driver.page_source, 'html.parser')
            commentBox = originalSoup
            time.sleep(5)
            button = driver.find_element_by_class_name('share-bar__list')
            try:
                commentButton = button.find_element_by_class_name('comments')
                commentButton = commentButton.find_element_by_tag_name('a')
            except:
                commentButton = button.find_element_by_class_name('share-bar__comments')
            commentButton.click()
            
            time.sleep(200)
            
            iframes = driver.find_elements_by_tag_name('iframe')
            #iframe = driver.find_elements_by_id('bb4b0ab3b34f383d8da32a70bad542e1-iframe')
            driver.switch_to.frame(iframes[0])
            newSoupArticle = BeautifulSoup(driver.page_source, 'html.parser')
            print(newSoupArticle)
            
            article_title.append(cleaned_title)
            article_URL.append(linkURL)

for x in range (2, 15):
    url = 'https://www.huffingtonpost.com/section/politics?page=' + str(x)
    scrapeTest(url)



/entry/joe-manchin-top-democrat-energy-committee_us_5c104c79e4b0ac537179c0af
us_5c104c79e4b0ac537179c0af
https://www.huffingtonpost.com/entry/joe-manchin-top-democrat-energy-committee_us_5c104c79e4b0ac537179c0af
https://m.huffpost.com/us/entry/us_5c104c79e4b0ac537179c0af/comments

Joe Manchin Named To Key Senate Energy Post Despite Opposition From Left


/entry/kevin-mccarthy-trump-investigating_us_5c0ed1dce4b06484c9fda235
us_5c0ed1dce4b06484c9fda235
https://www.huffingtonpost.com/entry/kevin-mccarthy-trump-investigating_us_5c0ed1dce4b06484c9fda235
https://m.huffpost.com/us/entry/us_5c0ed1dce4b06484c9fda235/comments

Kevin McCarthy Says Democrats Shouldn't Spend A Lot Of Time Investigating Trump


/entry/toledo-ohio-terror-attacks-suspects_us_5c0eb60de4b0edf5a3a7953b
us_5c0eb60de4b0edf5a3a7953b
https://www.huffingtonpost.com/entry/toledo-ohio-terror-attacks-suspects_us_5c0eb60de4b0edf5a3a7953b
https://m.huffpost.com/us/entry/us_5c0eb60de4b0edf5a3a7953b/comments

FBI Announces Arrests O

KeyboardInterrupt: 

In [ ]:
# Get time stamp and comment count of each article 
time_stamp = []
comment_count = []


def getTimeComments(url): 
    single_req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(single_req.text, 'html.parser')
    time = soup.find('span', {'class' : "timestamp__date--published"}).text
    time_stamp.append(time)
#   Where you would get the comments here
#   comments = soup.find('span', {'class' : "param-0 param-messagesCount"}).text??
#   comment_count.append(comments)
    
for link in article_URL: 
    getTimeComments(link)

In [83]:
# chrome_path = "./chromedriver"
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome('/Users/doreennguyen/Desktop/news-data-scrape/chromedriver')

In [84]:
# Set up dataframe 
d = {"title" : article_title, "url" : article_URL, "time_stamp" : time_stamp}
articleDF = pd.DataFrame(data=d)
articleDF

,title,url,time_stamp
0,fox news migrant invasion,https://www.huffingtonpost.com/entry/fox-news-...,12/09/2018 07:48 pm ET
1,fox news andrew mccarthy trump indicted,https://www.huffingtonpost.com/entry/fox-news-...,12/09/2018 04:04 pm ET
2,nick ayers decline trump chief staff,https://www.huffingtonpost.com/entry/nick-ayer...,12/09/2018 05:07 pm ET
3,rand paul william barr,https://www.huffingtonpost.com/entry/rand-paul...,12/09/2018 01:07 pm ET
4,trump first wanted his attorney general pick w...,https://www.huffingtonpost.com/entry/trump-fir...,12/09/2018 03:19 pm ET
5,jared kushner saudi crown prince khashoggi,https://www.huffingtonpost.com/entry/jared-kus...,12/09/2018 08:21 am ET
6,marco rubio trump manafort pardon,https://www.huffingtonpost.com/entry/marco-rub...,12/09/2018 11:27 am ET
7,tony evers scott walker legislature,https://www.huffingtonpost.com/entry/tony-ever...,12/09/2018 12:12 pm ET
8,cohen manafort trump nadler impeachment,https://www.huffingtonpost.com/entry/cohen-man...,12/09/2018 12:05 pm ET
9,trump bashed over twitter trolling obama for c...,https://www.huffingtonpost.com/entry/trump-bas...,12/08/2018 10:57 pm ET
